In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Crown-of-Thorns Starfish Detection Pipeline

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/models/blob/master/official/projects/cots_detector/crown_of_thorns_starfish_detection_pipeline.ipynb?force_crab_mode=1"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/models/blob/master/official/projects/cots_detector/crown_of_thorns_starfish_detection_pipeline.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
</table>

Coral reefs are some of the most diverse and important ecosystems in the world , however they face a number of rising threats that have resulted in massive global declines. In Australia, outbreaks of the coral-eating crown-of-thorns starfish (COTS) have been shown to cause major coral loss, with just 15 starfish in a hectare being able to strip a reef of 90% of its coral tissue. While COTS naturally exist in the Indo-Pacific, overfishing and excess run-off nutrients have led to massive outbreaks that are devastating already vulnerable coral communities.

Controlling COTS populations is critical to promoting coral growth and resilience, so Google teamed up with Australia’s national science agency, [CSIRO](https://www.csiro.au/en/), to tackle this problem. We trained ML object detection models to help scale underwater surveys, enabling the monitoring and mapping out these harmful invertebrates with the ultimate goal of helping control teams to address and prioritize outbreaks.

## About this notebook

This notebook tutorial shows how to detect COTS using a pre-trained COTS detector implemented in TensorFlow. On top of just running the model on each frame of the video, the tracking code in this notebook aligns detections from frame to frame creating a consistent track for each COTS. Each track is given an id and frame count. Here is an example image from a video of a reef showing labeled COTS starfish.

<img src="https://storage.googleapis.com/download.tensorflow.org/data/cots_detection/COTS_detected_sample.png">

It is recommended to enable GPU to accelerate the inference. On CPU, this runs for about 40 minutes, but on GPU it takes only 10 minutes. (In Colab it should already be set to GPU in the Runtime menu: *Runtime > Change runtime type > Hardware accelerator > select "GPU"*).

## Setup 

Install all needed packages.

In [ ]:
# remove the existing datascience package to avoid package conflicts in the colab environment
!pip3 uninstall -y datascience
!pip3 install -q opencv-python
!pip3 install PILLOW

In [ ]:
# Imports
import base64
import copy
import dataclasses
import glob
import logging
import mimetypes
import os
import pathlib
import subprocess
import time
import textwrap
from typing import Dict, Iterable, List, Optional, Tuple

from absl import logging as absl_logging
from IPython import display
import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import tensorflow as tf
from tqdm import tqdm

Define all needed variables.

In [ ]:
model_name = "cots_1080_v1" #@param ["cots_1080_v1", "cots_720_v1"]
test_sequence_name = "test3" #@param ["test1", "test2", "test3"]

In [ ]:
cots_model = f"https://storage.googleapis.com/download.tensorflow.org/models/cots_detection/{model_name}.zip"

# Alternatively, this dataset can be downloaded through CSIRO's Data Access Portal at https://data.csiro.au/collection/csiro:54830v2
sample_data_link = f"https://storage.googleapis.com/download.tensorflow.org/data/cots_detection/sample_images.zip"

preview_video_path = "preview.mp4"
detection_small_video_path = "COTS_detection.mp4"
detection_csv_path = "detections.csv"

You also need to retrieve the sample data.  This sample data is made up of a series of chronological images.

In [ ]:
sample_data_path = tf.keras.utils.get_file(origin=sample_data_link)
# Unzip data
!mkdir sample_images
!unzip -o -q {sample_data_path} -d sample_images

Convert the images to a video file:

In [ ]:
tmp_video_path = "tmp_preview.mp4"

filenames = sorted(glob.glob(f"sample_images/{test_sequence_name}/*.jpg"))
img = cv2.imread(filenames[0])
height, width, layers = img.shape
size = (width, height)

video_writer = cv2.VideoWriter(
    filename=tmp_video_path,
    fourcc=cv2.VideoWriter_fourcc(*"MP4V"), 
    fps=15, 
    frameSize=size)
 
for filename in tqdm(filenames):
    img = cv2.imread(filename)
    video_writer.write(img)
cv2.destroyAllWindows()
video_writer.release()

Re-encode the video, and reduce its size (Colab crashes if you try to embed the full size video).

In [ ]:
subprocess.check_call([
    "ffmpeg", "-y", "-i", tmp_video_path,
    "-vf","scale=800:-1",
    "-crf", "18",
    "-preset", "veryfast",
    "-vcodec", "libx264", preview_video_path])

The images you downloaded are frames of a movie showing a top view of a coral reef with crown-of-thorns starfish.  Use the `base64` data-URL trick to embed the video in this notebook:

In [ ]:
def embed_video_file(path: os.PathLike) -> display.HTML:
  """Embeds a file in the notebook as an html tag with a data-url."""
  path = pathlib.Path(path)
  mime, unused_encoding = mimetypes.guess_type(str(path))
  data = path.read_bytes()

  b64 = base64.b64encode(data).decode()
  return display.HTML(
      textwrap.dedent("""
      <video width="640" height="480" controls>
        <source src="data:{mime};base64,{b64}" type="{mime}">
        Your browser does not support the video tag.
      </video>
      """).format(mime=mime, b64=b64))


In [ ]:
embed_video_file(preview_video_path)

Can you se them? there are lots. The goal of the model is to put boxes around all of the starfish.  Each starfish will get its own ID, and that ID will be stable as the camera passes over it.

## Load the model

Download the trained COTS detection model that matches your preferences from earlier.

In [ ]:
model_path = tf.keras.utils.get_file(origin=cots_model)
# Unzip model
!mkdir {model_name}
!unzip -o -q {model_path} -d {model_name}

Load trained model from disk and create the inference function `model_fn()`.  This might take a little while.

In [ ]:
absl_logging.set_verbosity(absl_logging.ERROR)

tf.config.optimizer.set_experimental_options({'auto_mixed_precision': True})
tf.config.optimizer.set_jit(True)

model_fn = tf.saved_model.load(model_name).signatures['serving_default']

Here's one test image; how many COTS can you see?

In [ ]:
example_frame_number = 52
image = tf.io.read_file(filenames[example_frame_number])
image = tf.io.decode_jpeg(image)

# Caution PIL and tf use "RGB" color order, while cv2 uses "BGR".
PIL.Image.fromarray(image.numpy())

## Raw model outputs

Try running the model on the image. The model expects a batch of images so add an outer `batch` dimension before calling the model.

Note: The model only runs correctly with a batch size of 1.

The result is a dictionary with a number of fields. For all fields the first dimension of the shape is the `batch` dimension, 

In [ ]:
image_batch = image[tf.newaxis, ...]
result = model_fn(image_batch)

print(f"{'image_batch':20s}- shape: {image_batch.shape}")

for key, value in result.items():
  print(f"{key:20s}- shape: {value.shape}")

The `num_detections` field gives the number of valid detections, but this is always 100. There are always 100 locations that _could_ be a COTS.

In [ ]:
print('\nnum_detections: ', result['num_detections'].numpy())

Similarly the `detection_classes` field is always `0`, since the model only detects 1 class: COTS.

In [ ]:
print('detection_classes: \n', result['detection_classes'].numpy())

What actually matters here is the detection scores, indicating the quality of each detection: 

In [ ]:
result['detection_scores'].numpy()

You need to choose a threshold that determines what counts as a good detection. This frame has a few good detections:

In [ ]:
good_detections = result['detection_scores'] > 0.4
good_detections.numpy()

## Bounding boxes and detections

Build a class to handle the detection boxes:

In [ ]:
@dataclasses.dataclass(frozen=True)
class BBox:
  x0: float
  y0: float
  x1: float
  y1: float

  def replace(self, **kwargs):
    d = self.__dict__.copy()
    d.update(kwargs)
    return type(self)(**d)

  @property
  def center(self)-> Tuple[float, float]:
    return ((self.x0+self.x1)/2, (self.y0+self.y1)/2)
  
  @property
  def width(self) -> float:
    return self.x1 - self.x0

  @property
  def height(self) -> float:
    return self.y1 - self.y0

  @property
  def area(self)-> float:
    return (self.x1 - self.x0 + 1) * (self.y1 - self.y0 + 1)
  
  def intersection(self, other)-> Optional['BBox']:
    x0 = max(self.x0, other.x0)
    y0 = max(self.y0, other.y0)
    x1 = min(self.x1, other.x1)
    y1 = min(self.y1, other.y1)
    if x0 > x1 or y0 > y1:
      return None
    return BBox(x0, y0, x1, y1)

  def iou(self, other):
    intersection = self.intersection(other)
    if intersection is None:
      return 0
    
    ia = intersection.area

    return ia/(self.area + other.area - ia)
  
  def draw(self, image, label=None, color=(0, 140, 255)):
    image = np.asarray(image)
    cv2.rectangle(image, 
                  (int(self.x0), int(self.y0)),
                  (int(self.x1), int(self.y1)),
                  color,
                  thickness=2)
    if label is not None:
      cv2.putText(image, str(label), 
                  (int(self.x0), int(self.y0-10)),
                  cv2.FONT_HERSHEY_SIMPLEX,
                  0.9, color, thickness=2)
    return image

And a class to represent a `Detection`, with a method to create a list of detections from the model's output:

In [ ]:
@dataclasses.dataclass(frozen=True)
class Detection:
    """Detection dataclass."""
    class_id: int
    score: float
    bbox: BBox
    threshold:float = 0.4

    def replace(self, **kwargs):
      d = self.__dict__.copy()
      d.update(kwargs)
      return type(self)(**d)

    @classmethod
    def process_model_output(
      cls, image, detections: Dict[str, tf.Tensor]
    ) -> Iterable['Detection']:
      
      # The model only works on a batch size of 1.
      detection_boxes = detections['detection_boxes'].numpy()[0]
      detection_classes = detections['detection_classes'].numpy()[0].astype(np.int32)
      detection_scores = detections['detection_scores'].numpy()[0]

      img_h, img_w = image.shape[0:2]

      valid_indices = detection_scores >= cls.threshold
      classes = detection_classes[valid_indices]
      scores = detection_scores[valid_indices]
      boxes = detection_boxes[valid_indices, :]
      detections = []

      for class_id, score, box in zip(classes, scores, boxes):
          detections.append(
              Detection(
                  class_id=class_id,
                  score=score,
                  bbox=BBox(
                      x0=box[1] * img_w,
                      y0=box[0] * img_h,
                      x1=box[3] * img_w,
                      y1=box[2] * img_h,)))

      return detections

## Preview some detections

Now you can preview the model's output:

In [ ]:
detections = Detection.process_model_output(image, result)

for n, det in enumerate(detections):
  det.bbox.draw(image, label=n+1, color=(255, 140, 0))

PIL.Image.fromarray(image.numpy())

That works well for one frame, but to count the number of COTS in a video you'll need to track the detections from frame to frame. The raw detection indices are not stable, they're just sorted by the detection score. Below both sets of detections are overlaid on the second image with the first frame's detections in white and the second frame's in orange, the indices are not aligned. The positions are shifted because of camera motion between the two frames:

In [ ]:
image2 = tf.io.read_file(filenames[example_frame_number+5]) # five frames later
image2 = tf.io.decode_jpeg(image2)
result2 = model_fn(image2[tf.newaxis, ...])
detections2 = Detection.process_model_output(image2, result2)

for n, det in enumerate(detections):
  det.bbox.draw(image2, label=n+1, color=(255, 255, 255))

for n, det in enumerate(detections2):
  det.bbox.draw(image2, label=n+1, color=(255, 140, 0))

PIL.Image.fromarray(image2.numpy())

## Use optical flow to align detections

The two sets of bounding boxes above don't line up because of camera movement. 
To see in more detail how tracks are aligned, initialize the tracker with the first image, and then run the optical flow step, `propagate_tracks`. 

In [ ]:
def default_of_params():
  its=20
  eps=0.03
  return {
      'winSize': (64,64),
      'maxLevel': 3,
      'criteria': (cv2.TermCriteria_COUNT + cv2.TermCriteria_EPS, its, eps)
  }

In [ ]:
def propagate_detections(detections, image1, image2, of_params=None):
  if of_params is None:
    of_params = default_of_params()

  bboxes = [det.bbox for det in detections]
  centers = np.float32([[bbox.center for bbox in bboxes]])
  widths = np.float32([[bbox.width for bbox in bboxes]])
  heights = np.float32([[bbox.height for bbox in bboxes]])


  new_centers, status, error = cv2.calcOpticalFlowPyrLK(
      image1, image2, centers, None, **of_params)

  x0s = new_centers[...,0] - widths/2
  x1s = new_centers[...,0] + widths/2
  y0s = new_centers[...,1] - heights/2
  y1s = new_centers[...,1] + heights/2

  updated_detections = []
  for i, det in enumerate(detections):
    det = det.replace(
        bbox = BBox(x0s[0,i], y0s[0,i], x1s[0,i], y1s[0,i]))
    updated_detections.append(det)
  return updated_detections

Now keep the white boxes for the initial detections, and the orange boxes for the new set of detections. But add the optical-flow propagated tracks in green. You can see that by using optical-flow to propagate the old detections to the new frame the alignment is quite good. It's this alignment between the old and new detections (between the green and orange boxes) that allows the tracker to make a persistent track for each COTS.  

In [ ]:
image = tf.io.read_file(filenames[example_frame_number])
image = tf.io.decode_jpeg(image).numpy()
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

image2 = tf.io.read_file(filenames[example_frame_number+5]) # five frames later
image2 = tf.io.decode_jpeg(image2).numpy()
image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

updated_detections = propagate_detections(detections, image_gray, image2_gray)


for det in detections:
  det.bbox.draw(image2, color=(255, 255, 255))

for det in updated_detections:
  det.bbox.draw(image2, color=(0, 255, 0))

for det in detections2:
  det.bbox.draw(image2, color=(255, 140, 0))

PIL.Image.fromarray(image2)

## Define **OpticalFlowTracker** class

These help track the movement of each COTS object across the video frames.

The tracker collects related detections into `Track` objects. 

The class's init is defined below, it's methods are defined in the following cells.

The `__init__` method just initializes the track counter (`track_id`), and sets some default values for the tracking and optical flow configurations. 

In [ ]:
class OpticalFlowTracker:
    """Optical flow tracker."""

    @classmethod
    def add_method(cls, fun):
      """Attach a new method to the class."""
      setattr(cls, fun.__name__, fun)


    def __init__(self, tid=1, ft=3.0, iou=0.5, tt=2.0, bb=32, of_params=None):
      # Bookkeeping for the tracks.
      # The running track count, incremented for each new track.
      self.track_id = tid
      self.tracks = []
      self.prev_image = None
      self.prev_time = None

      # Configuration for the track cleanup logic.
      # How long to apply optical flow tracking without getting positive 
      # detections (sec).
      self.track_flow_time = ft * 1000
      # Required IoU overlap to link a detection to a track.
      self.overlap_threshold = iou
      # Used to detect if detector needs to be reset.
      self.time_threshold = tt * 1000
      self.border = bb

      if of_params is None:
        of_params = default_of_params()
      self.of_params = of_params


Internally the tracker will use small `Track` and `Tracklet` classes to organize the data. The `Tracklet` class is just a `Detection` with a timestamp, while a `Track` is a track ID, the most recent detection and a list of `Tracklet` objects forming the history of the track.

In [ ]:
@dataclasses.dataclass(frozen=True)
class Tracklet:
    timestamp:float
    detection:Detection

    def replace(self, **kwargs):
      d = self.__dict__.copy()
      d.update(kwargs)
      return type(self)(**d)

In [ ]:
@dataclasses.dataclass(frozen=True)
class Track:
    """Tracker entries."""
    id:int
    det: Detection
    linked_dets:List[Tracklet] = dataclasses.field(default_factory=list)

    def replace(self, **kwargs):
      d = self.__dict__.copy()
      d.update(kwargs)
      return type(self)(**d)

The tracker keeps a list of active `Track` objects.

The main `update` method takes an image, along with the list of detections and the timestamp for that image. On each frame step it performs the following sub-tasks:

* The tracker uses optical flow to calculate where each `Track` expects to see a new `Detection`.
* The tracker matches up the actual detections for the frame to the expected detections for each Track.
* If a detection doesn't get matched to an existing track, a new track is created for the detection.
* If a track stops getting assigned new detections, it is eventually deactivated. 

In [ ]:
@OpticalFlowTracker.add_method
def update(self, image_bgr, detections, timestamp):
    start = time.time()

    image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)

    # Remove dead tracks.
    self.tracks = self.cleanup_tracks(image, timestamp)

    # Run optical flow to update existing tracks.
    if self.prev_time is not None:
        self.tracks = self.propagate_tracks(image)

    # Update the track list based on the new detections
    self.apply_detections_to_tracks(image, detections, timestamp)

    self.prev_image = image
    self.prev_time = timestamp

    return self.tracks

The `cleanup_tracks` method clears tracks that are too old or are too close to the edge of the image.

In [ ]:
@OpticalFlowTracker.add_method
def cleanup_tracks(self, image, timestamp) -> List[Track]:
  image_w = image.shape[1]
  image_h = image.shape[0]

  # Assume tracker is invalid if too much time has passed!
  if (self.prev_time is not None and
          timestamp - self.prev_time > self.time_threshold):
      logging.info(
          'Too much time since last update, resetting tracker.')
      return []

  # Remove tracks which are:
  # - Touching the image edge.
  # - Have existed for a long time without linking a real detection.
  active_tracks = []
  for track in self.tracks:
      bbox = track.det.bbox
      if (bbox.x0 < self.border or bbox.y0 < self.border or
              bbox.x1 >= (image_w - self.border) or
              bbox.y1 >= (image_h - self.border)):
          logging.info(f'Removing track {track.id} because it\'s near the border')
          continue

      time_since_last_detection = timestamp - track.linked_dets[-1].timestamp
      if (time_since_last_detection > self.track_flow_time):
          logging.info(f'Removing track {track.id} because it\'s too old '
                        f'({time_since_last_detection:.02f}s)')
          continue

      active_tracks.append(track)

  return active_tracks

The `propagate_tracks` method uses optical flow to update each track's bounding box's position to predict their location in the new image: 

In [ ]:
@OpticalFlowTracker.add_method
def propagate_tracks(self, image):
  if not self.tracks:
    return self.tracks[:]

  detections = [track.det for track in self.tracks]
  detections = propagate_detections(detections, self.prev_image, image, self.of_params)

  return [track.replace(det=det) 
          for track, det in zip(self.tracks, detections)]


The `apply_detections_to_tracks` method compares each detection to the updated bounding box for each track. The detection is added to the track that matches best, if the match is better than the `overlap_threshold`. If no track is better than the threshold, the detection is used to create a new track. 

If a track has no new detection assigned to it the predicted detection is used.

In [ ]:
@OpticalFlowTracker.add_method
def apply_detections_to_tracks(self, image, detections, timestamp):
  image_w = image.shape[1]
  image_h = image.shape[0]

  # Insert new detections.
  detected_obj_track_ids = set()

  for detection in detections:
      bbox = detection.bbox
      if (bbox.x0 < self.border or bbox.y0 < self.border or
              bbox.x1 >= image_w - self.border or
              bbox.y1 >= image_h - self.border):
          logging.debug('Skipping detection because it\'s close to the border.')
          continue

      # See if detection can be linked to an existing track.
      linked = False
      overlap_index = 0
      overlap_max = -1000
      for track_index, track in enumerate(self.tracks):
          logging.debug('Testing track %d', track_index)
          if track.det.class_id != detection.class_id:
              continue
          overlap = detection.bbox.iou(track.det.bbox)
          if overlap > overlap_max:
              overlap_index = track_index
              overlap_max = overlap

      # Link to existing track with maximal IoU.
      if overlap_max > self.overlap_threshold:
          track = self.tracks[overlap_index]
          self.tracks[overlap_index] = track.replace(det=detection)
          track.linked_dets.append(Tracklet(timestamp, detection))
          detected_obj_track_ids.add(track.id)
          linked = True

      if not linked:
          logging.info(f'Creating new track with ID {self.track_id}')
          new_track = Track(self.track_id, detection)
          new_track.linked_dets.append(Tracklet(timestamp, detection))
          detected_obj_track_ids.add(self.track_id)
          self.tracks.append(new_track)
          self.track_id += 1

  for track in self.tracks:
      # If the detector does not find the obj but estimated in the tracker, 
      # add the estimated one to that tracker's linked_dets
      if track.id not in detected_obj_track_ids:
          track.linked_dets.append(Tracklet(timestamp, track.det))

## Test run the tracker

So reload the test images, and run the detections to test out the tracker.

On the first frame it creates and returns one track per detection:

In [ ]:
example_frame_number = 52
image = tf.io.read_file(filenames[example_frame_number])
image = tf.io.decode_jpeg(image)
result = model_fn(image[tf.newaxis, ...])
detections = Detection.process_model_output(image, result)

tracker = OpticalFlowTracker()
tracks = tracker.update(image.numpy(), detections, timestamp = 0)

print(f'detections : {len(detections)}') 
print(f'tracks     : {len(tracks)}')

On the second frame many of the detections get assigned to existing tracks:

In [ ]:
image2 = tf.io.read_file(filenames[example_frame_number+5]) # five frames later
image2 = tf.io.decode_jpeg(image2)
result2 = model_fn(image2[tf.newaxis, ...])
detections2 = Detection.process_model_output(image2, result2)

new_tracks = tracker.update(image2.numpy(), detections2, timestamp = 1000)

print(f'detections : {len(detections2)}') 
print(f'tracks     : {len(new_tracks)}')

Now the track IDs should be consistent:

In [ ]:
test_img = image2.numpy()
for n,track in enumerate(tracks):
  track.det.bbox.draw(test_img, label=n, color=(255, 255, 255))

for n,track in enumerate(new_tracks):
  track.det.bbox.draw(test_img, label=n, color=(255, 140, 0))

PIL.Image.fromarray(test_img)

## Perform the COTS detection inference and tracking.

The main tracking loop will perform the following: 

1. Load the images in order.
2. Run the model on the image.
3. Update the tracker with the new images and detections.
4. Keep information about each track (id, current index and length) analysis or display. 

The `TrackAnnotation` class, below, will collect the data about each track:

In [ ]:
@dataclasses.dataclass(frozen=True)
class TrackAnnotation:
  det: Detection
  seq_id: int
  seq_idx: int
  seq_length: Optional[int] = None

  def replace(self, **kwargs):
    d = self.__dict__.copy()
    d.update(kwargs)
    return type(self)(**d)

  def annotation_str(self):
    return f"{self.seq_id} ({self.seq_idx}/{self.seq_length})"


The `parse_image` function, below, will take `(index, filename)` pairs load the images as tensors and return `(timestamp_ms, filename, image)` triples, assuming 30fps

In [ ]:
# Read a jpg image and decode it to a uint8 tf tensor.
def parse_image(index, filename):
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    timestamp_ms = 1000*index/30 # assuming 30fps
    return (timestamp_ms, filename, image)

Here is the main tracker loop. Note that initially the saved `TrackAnnotations` don't contain the track lengths. The lengths are collected in the `track_length_for_id` dict.

In [ ]:
# Create a tracker object
tracker = OpticalFlowTracker(tid=1)
# Record tracking responses from the tracker
detection_result = []
# Record the length of each tracking sequence
track_length_for_id = {}

# Create a data loader
file_list = sorted(glob.glob(f"sample_images/{test_sequence_name}/*.jpg"))
list_ds = tf.data.Dataset.from_tensor_slices(file_list).enumerate()
images_ds = list_ds.map(parse_image)

# Traverse the dataset with batch size = 1, you cannot change the batch size
for timestamp_ms, file_path, images in tqdm(images_ds.batch(1, drop_remainder=True)):
    # get detection result
    detections = Detection.process_model_output(images[0], model_fn(images))

    # Feed detection results and the corresponding timestamp to the tracker, and then get tracker response
    tracks = tracker.update(images[0].numpy(), detections, timestamp_ms[0])
    annotations = []
    for track in tracks:
      anno = TrackAnnotation(
          det=track.det,
          seq_id = track.id,
          seq_idx = len(track.linked_dets)
      )
      annotations.append(anno)
      track_length_for_id[track.id] = len(track.linked_dets)
    
    detection_result.append((file_path.numpy()[0].decode(), annotations))

Once the tracking loop has completed you can update the track length (`seq_length`) for each annotation from the `track_length_for_id` dict:

In [ ]:
def update_annotation_lengths(detection_result, track_length_for_id):
  new_result = []
  for file_path, annotations in detection_result:
    new_annotations = []
    for anno in annotations:
      anno = anno.replace(seq_length=track_length_for_id[anno.seq_id])
      new_annotations.append(anno)
    new_result.append((file_path, new_annotations))
  return new_result

In [ ]:
detection_result = update_annotation_lengths(detection_result, track_length_for_id)

## Output the detection results and play the result video

Once the inference is done, we draw the bounding boxes and track information onto each frame's image. Finally, we combine all frames into a video for visualisation.

In [ ]:
detection_full_video_path = "COTS_detection_full_size.mp4"
detect_video_writer = cv2.VideoWriter(
    filename=detection_full_video_path,
    fourcc=cv2.VideoWriter_fourcc(*"MP4V"), 
    fps=15, 
    frameSize=size)

for file_path, annotations in tqdm(detection_result):
    image = cv2.imread(file_path)
    for anno in annotations:
      anno.det.bbox.draw(image, label=anno.annotation_str(), color=(0, 140, 255))
    detect_video_writer.write(image)
cv2.destroyAllWindows()

detect_video_writer.release()

In [ ]:
subprocess.check_call([
    "ffmpeg","-y", "-i", detection_full_video_path,
    "-vf","scale=800:-1",
    "-crf", "18",
    "-preset", "veryfast",
    "-vcodec", "libx264", detection_small_video_path])

In [ ]:
embed_video_file(detection_small_video_path)

The output video is now saved as movie at `detection_full_video_path`. You can download your video by uncommenting the following code.

In [ ]:
#try:
#  from google.colab import files
#  files.download(detection_full_video_path)
#except ImportError:
#  pass